In [23]:
print("Is the kernel alive?")
print("Yes, the kernel is alive")

Is the kernel alive?
Yes, the kernel is alive


In [25]:
# First lets install smolagents
# %pip install -q -U smolagents
# %pip install -q -U ipywidgets

In [ ]:
# Login to Hugging Face Either use the notebook_login, command line or load the .env file
# from huggingface_hub import notebook_login
# notebook_login()

# Load the .env file, the following are loaded from the .env file
# 1. HF_TOKEN

from dotenv import load_dotenv
load_dotenv()


In [ ]:
#@title Connecting to LM Studio
import requests

# Define LM Studio endpoint
LM_STUDIO_URL = "http://127.0.0.1:1234"

# Test that LM Studio is working
try:
    response = requests.get(f"{LM_STUDIO_URL}/v1/models")
    if response.status_code == 200:
        print("✅ LM Studio is running!")
        print("Available models:", response.json())
    else:
        print("⚠️ Unable to reach LM Studio. Make sure it's running.")
except requests.exceptions.ConnectionError:
    print("❌ LM Studio is not running. Start it and try again.")

In [22]:
prompt = "What is the the best fruit? Choose one."

llm_model = "deepseek-r1-distill-qwen-14b"
max_tokens = 4096
temperature = 0.6
# Define the request payload for LM Studio
payload = {
    "model": llm_model,
    "promt": prompt,
    "messages": [
        {"role": "user", "content": prompt}
        ],
    "max_tokens": max_tokens,
    "temperature": temperature
}

# Send the request to LM Studio
response = requests.post(f"{LM_STUDIO_URL}/v1/chat/completions", json=payload)

# Parse and print the response

if response.status_code == 200:
    result = response.json()
    print("🤖 LM Studio:", result["choices"][0]["message"]["content"])  # Fixed key
else:
    print("❌ Something went wrong:", response.text)

NameError: name 'requests' is not defined

In [ ]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, HfApiModel

agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=HfApiModel())

agent.run("Search for the best music recommendations for a party at the Wayne's mansion.")

In [ ]:
# %pip install -q -U smolagents[all] requests
from smolagents import CodeAgent
import requests

# Configuration
LM_STUDIO_HOST = "http://127.0.0.1:1234"
MODEL_NAME = "deepseek-r1-distill-qwen-14b"
API_ENDPOINT = f"{LM_STUDIO_HOST}/v1/chat/completions"

class LMStudioModel:
    def __init__(self, model_name):
        self.model_name = model_name
        
    def generate(self, messages, **kwargs):
        """Handle LM Studio API calls with error handling"""
        try:
            response = requests.post(
                API_ENDPOINT,
                json={
                    "model": self.model_name,
                    "messages": messages,
                    "max_tokens": kwargs.get("max_tokens", 4096),
                    "temperature": kwargs.get("temperature", 0.6)
                },
                timeout=30
            )
            response.raise_for_status()
            return response.json()['choices'][0]['message']['content']
        except Exception as e:
            return f"API Error: {str(e)}"

# Initialize agent
local_agent = CodeAgent(
    tools=[],
    model=LMStudioModel(MODEL_NAME),
    add_base_tools=False
)

# Test run
if __name__ == "__main__":
    print(f"Using local model: {MODEL_NAME}")
    response = local_agent.run("What's the best fruit? Choose one and explain briefly.")
    print("Agent Response:", response)

In [20]:
# Using LM Studio with LiteLLM https://docs.litellm.ai/docs/providers/lm_studio

import os
from dotenv import load_dotenv

load_dotenv()

# env variables
os.environ["LM_STUDIO_API_BASE"]
os.environ["LM_STUDIO_API_KEY"]

from litellm import completion
import os

# Variables
local_model = "lm_studio/deepseek-r1-distill-qwen-14b"

response = completion(
    model=local_model,
    messages=[
        {
            "role": "user",
            "content": "What's the weather like in Boston today in Fahrenheit?",
        }
    ]
)
print(response)


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers



AuthenticationError: litellm.AuthenticationError: AuthenticationError: Lm_studioException - Error code: 401 - {'error': {'message': 'Incorrect API key provided: lm-studi*****-key. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [7]:
from smolagents import CodeAgent, LiteLLMModel, DuckDuckGoSearchTool

# Configure LM Studio connection via LiteLLM
local_model_id = "lm_studio/deepseek-r1-distill-qwen-14b"
LM_STUDIO_HOST = "http://127.0.0.1:1234"

local_model = LiteLLMModel(
    model_id=local_model_id,
    api_base=LM_STUDIO_HOST,
    api_key="lm-studio-api-key" # not actually needed for local model
)

local_agent = CodeAgent(
    tools=[DuckDuckGoSearchTool()],
    model=local_model,
    additional_authorized_imports=['requests','json'],
)

local_agent.run("What is my name?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is my name?                                                                                                │
│                                                                                                                 │
╰─ LiteLLMModel - lm_studio/deepseek-r1-distill-qwen-14b ─────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



Error in generating model output:
litellm.BadRequestError: Lm_studioException - 

[Step 0: Duration 0.01 seconds]

...

[Step 5: Duration 0.02 seconds]


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



Reached max steps.

[Step 6: Duration 0.02 seconds]

'Error in generating final LLM output:\nlitellm.BadRequestError: Lm_studioException - '

In [ ]:
agent.run("Reproduce algunas canciones de Juan Gabriel")

In [ ]:
from smolagents import CodeAgent, tool

@tool
def suggest_menu(occasion: str) -> str:
    """
    Suggests a menu based on the occasion.
    Args:
        occasion: The type of occasion for the party.
    """
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "Custom menu for the butler."
    
agent = CodeAgent(
    tools=[suggest_menu],
    model=HfApiModel(),
)

agent.run("Prepare a formal menu for today's dinner.")

In [ ]:
agent.run("Prepara una list de canciones de Juan Gabriel para escuchar en la fiesta.")

In [ ]:
# Some additional permisions can be added to the agent
from smolagents import CodeAgent, HfApiModel
import numpy as np
import time
import datetime

agent = CodeAgent(
    tools=[],
    model=HfApiModel(),
    additional_authorized_imports=['datetime'],
)

prompt = """Alfred needs to prepare for the party. Here are the tasks:
1. Prepare the drinks - 30 minutes
2. Decorate the mansion - 60 minutes
3. Set up the menu - 45 minutes
4. Prepare the music and playlsit - 45 minutes

If we start right tomorrow at 5 pm, at what time will the party be ready?
"""
agent.run(prompt)

In [ ]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, HfApiModel, VisitWebpageTool, FinalAnswerTool, Tool, tool

@tool
def suggest_menu(occasion: str) -> str:
    """
    Suggests a menu based on the occasion.
    Args:
        occasion: The type of occasion for the party.
    """
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "Custom menu for the butler."

@tool
def catering_service_tool(query: str) -> str:
    """
    This tool returns the highest-rated catering service in Gotham City.

    Args:
        query: A search term for finding catering services.
    """
    # Example list of catering services and their ratings
    services = {
        "Gotham Catering Co.": 4.9,
        "Wayne Manor Catering": 4.8,
        "Gotham City Events": 4.7,
    }

    # Find the highest rated catering service (simulating search query filtering)
    best_service = max(services, key=services.get)

    return best_service

class SuperheroPartyThemeTool(Tool):
    name = "superhero_party_theme_generator"
    description = """
    This tool suggests creative superhero-themed party ideas based on a category.
    It returns a unique party theme idea."""

    inputs = {
        "category": {
            "type": "string",
            "description": "The type of superhero party (e.g., 'classic heroes', 'villain masquerade', 'futuristic Gotham').",
        }
    }

    output_type = "string"

    def forward(self, category: str):
        themes = {
            "classic heroes": "Justice League Gala: Guests come dressed as their favorite DC heroes with themed cocktails like 'The Kryptonite Punch'.",
            "villain masquerade": "Gotham Rogues' Ball: A mysterious masquerade where guests dress as classic Batman villains.",
            "futuristic Gotham": "Neo-Gotham Night: A cyberpunk-style party inspired by Batman Beyond, with neon decorations and futuristic gadgets."
        }

        return themes.get(category.lower(), "Themed party idea not found. Try 'classic heroes', 'villain masquerade', or 'futuristic Gotham'.")


# Alfred, the butler, preparing the menu for the party
agent = CodeAgent(
    tools=[
        DuckDuckGoSearchTool(),
        VisitWebpageTool(),
        suggest_menu,
        catering_service_tool,
        SuperheroPartyThemeTool()
        ],
    model=HfApiModel(),
    max_steps=10,
    verbosity_level=2
)

agent.run("Give me best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme")


In [ ]:
import io
import sys
# Add encoding fix before pushing
# sys.stdout = io.TextIOWrapper(sys.stdout.buffer, encoding='utf-8', errors='replace')
import os
os.environ["HF_HUB_DISABLE_EMOJIS"] = "1"
agent.push_to_hub('angelUndeveloped/AlfredAgentTutorialAYMS')

In [ ]:
agent = CodeAgent(tools=[], model=HfApiModel())
alfred_agent = agent.from_hub('angelUndeveloped/AlfredAgentTutorialAYMS', trust_remote_code=True)

In [ ]:
alfred_agent.run("Give me best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme")